In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import random

In [2]:
train_data = pd.read_csv('Data/train.csv')
train_data.set_index('id', inplace=True)
train_data = train_data.replace({'A': 0, 'B': 1})
train_data.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,1,0,0,0,0,1,0,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
2,0,1,0,0,0,0,0,0,1,1,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
5,0,1,0,0,1,0,0,0,1,1,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
10,1,1,0,1,0,0,0,0,1,0,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
11,0,1,0,1,0,0,0,0,1,1,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [3]:
test_data = pd.read_csv('Data/test.csv')
test_data.set_index('id', inplace=True)
test_data = test_data.replace({'A': 0, 'B': 1})
test_data.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,,,,,,,,
4,0,1,0,0,0,0,0,0,1,0,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
6,0,1,0,1,0,0,0,0,1,0,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
9,0,1,0,1,1,0,1,0,1,1,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
12,0,0,0,0,1,0,0,0,0,0,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
15,1,0,0,0,0,1,0,0,0,0,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


### Data Generation

In [129]:
columns = ['location', 'severity','ambulance', 'drunk driver', 'pain level', 'emotional distress', \
           'passengers', 'at fault', 'accident type', 'roll over', 'claim']
location = ['intersection', 'highway', 'stoplight']
accident_type = ['t-bone', 'head-on', 'rear-end']

In [130]:
data = pd.DataFrame(columns = columns)

In [131]:
for i in range(1001):
    temp = {'location': location[random.randint(0, 2)], 'severity': 0, 
            'ambulance': random.choice([0, 1]), 'drunk driver': random.choice([0, 1]),
            'pain level': round(random.random(), 4), 'emotional distress': round(random.random(), 4),
             'passengers': random.randint(0, 5), 'at fault': random.choice([0, 1]), 'accident type':
             accident_type[random.randint(0, 2)], 'roll over': random.choice([0, 1])}
    data.loc[i] = pd.Series(temp)  

In [132]:
for col in ['location', 'accident type']:
    data = pd.concat([data, pd.get_dummies(data[col], prefix = col)], axis = 1).drop(col, axis = 1)

In [180]:
weights = {'severity': 5000, 'ambulance': 2000, 'drunk driver': 20000, 'pain level': 2000,
           'emotional distress': 500, 'passengers': 100, 'at fault': -10000, 'roll over': 5000,
           'location_highway': 3000, 'location_intersection': 3000, 'location_stoplight': 1500,
           'accident type_head-on': 7000, 'accident type_rear-end': 2000, 'accident type_t-bone': 5000}

In [194]:
for i in range(1001):
    data.loc[i,'claim'] = (data.loc[i,'severity'])*(weights['severity']*random.gauss(1, 0)) + \
                           (data.loc[i,'ambulance'])*(weights['ambulance']) + \
                           (data.loc[i,'drunk driver'])*(weights['drunk driver']*random.gauss(1, 0)) + \
                           (data.loc[i,'pain level'])*(weights['pain level']*random.gauss(1, 0)) + \
                           (data.loc[i,'emotional distress'])*(weights['emotional distress']*random.gauss(1, 0)) + \
                           (data.loc[i,'passengers'])*(weights['passengers']*random.gauss(1, 0)) + \
                           (data.loc[i,'at fault'])*(weights['at fault']*random.gauss(1, 0)) + \
                           (data.loc[i,'roll over'])*(weights['roll over']*random.gauss(1, 0)) + \
                           (data.loc[i,'location_highway'])*(weights['location_highway']*random.gauss(1, 0)) + \
                           (data.loc[i,'location_intersection'])*(weights['location_intersection']*random.gauss(1, 0)) + \
                           (data.loc[i,'location_stoplight'])*(weights['location_stoplight']*random.gauss(1, 0)) + \
                           (data.loc[i,'accident type_head-on'])*(weights['accident type_head-on']*random.gauss(1, 0)) + \
                           (data.loc[i,'accident type_rear-end'])*(weights['accident type_rear-end']*random.gauss(1, 0)) + \
                           (data.loc[i,'accident type_t-bone'])*(weights['accident type_t-bone']*random.gauss(1, 0))
                                                     
                                                     
                                                     

In [195]:
final_data_df = data.clip(lower = 0)
final_data_df.head()

,severity,ambulance,drunk driver,pain level,emotional distress,passengers,at fault,roll over,claim,location_highway,location_intersection,location_stoplight,accident type_head-on,accident type_rear-end,accident type_t-bone
0,0,1,0,0.6175,0.0884,5,1,1,2279.20,0,0,1,0,1,0
1,0,1,0,0.5359,0.6841,1,1,0,0.00,1,0,0,0,1,0
2,0,0,0,0.5209,0.9816,1,1,1,3132.60,0,0,1,0,0,1
3,0,0,0,0.2439,0.1944,1,0,0,10685.00,1,0,0,1,0,0
4,0,1,0,0.4896,0.4147,1,1,1,6286.55,1,0,0,0,0,1


### Model Training

In [196]:
import math
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [197]:
X_train, X_test, y_train, y_test = train_test_split(final_data_df.drop(['claim'], axis = 1),
                                   final_data_df['claim'], random_state = 42, test_size = .2)

In [198]:
ols = linear_model.LinearRegression()
model = ols.fit(X_train, y_train)

In [207]:
pred = model.predict(X_test)
rmse = math.sqrt(mean_squared_error(y_test, pred))
rmse

509.61993153334447

In [208]:
r2 = r2_score(y_test, pred)
r2

0.9977453200191087

In [209]:
def predict_claim(lst):
    client_data = np.array(lst)
    prediction = model.predict(data-test.reshape(1, -1))
    if prediction <= 0:
        return 0
    else:
        return prediction